# Proyecto de Analisis de Datos

## Importaciones

In [ ]:
import pandas as pd

## Lectura

In [ ]:
df_init = pd.read_csv('https://raw.githubusercontent.com/LotusZaheer/Soil-data-analysis/main/Data/Resultados_de_An_lisis_de_Laboratorio_Suelos_en_Colombia.csv')